<a href="https://colab.research.google.com/github/ALexDPotan/ALexDPotan/blob/main/z_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Cell 1: Environment Setup & Dependencies**

In [ ]:
# Check GPU allocation
!nvidia-smi

# Install required packages
!pip install -q pyngrok flask

# Import environment management
import os
from google.colab import userdata, drive

**Cell 2: Ngrok Authentication (As Requested**

In [ ]:
# Securely load ngrok auth token from Colab secrets
os.environ["NGROK_AUTHTOKEN"] = userdata.get("NGROK_AUTHTOKEN")

# Alternative method: Manual input (uncomment if not using secrets)
# from getpass import getpass
# os.environ["NGROK_AUTHTOKEN"] = getpass("Enter your ngrok authtoken: ")

# Set pyngrok default configuration
from pyngrok import conf
conf.get_default().auth_token = os.environ["NGROK_AUTHTOKEN"]

print("✓ Ngrok authentication configured")

**Cell 3: Install ComfyUI**

In [ ]:
# Clone ComfyUI repository
%cd /content
!git clone https://github.com/comfyanonymous/ComfyUI
%cd ComfyUI

# Install ComfyUI dependencies
!pip install -q -r requirements.txt

# Create necessary directories
!mkdir -p models/checkpoints models/vae models/loras models/controlnet models/ipadapter models/clip_vision models/upscale_models custom_nodes

print("✓ ComfyUI installed successfully")

**Cell 4 :Install ComfyUI Manager**

In [ ]:
# Install ComfyUI Manager custom node
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git

# Install Manager dependencies
%cd ComfyUI-Manager
!pip install -q -r requirements.txt

print("✓ ComfyUI Manager installed successfully")
print("  Restart ComfyUI after installation to see the Manager button")

**Cell 5: Launch ComfyUI with Ngrok Tunnel**

In [ ]:
import threading
import time
from pyngrok import ngrok

# Kill any existing processes
!pkill -f "python main.py" 2>/dev/null
!pkill -f ngrok 2>/dev/null
time.sleep(2)

# Function to run ComfyUI
def run_comfyui():
    %cd /content/ComfyUI
    !python main.py --dont-print-server --listen 0.0.0.0 --port 8188

# Start ComfyUI in background thread
comfy_thread = threading.Thread(target=run_comfyui, daemon=True)
comfy_thread.start()

# Wait for server to initialize
print("⏳ Starting ComfyUI server...")
time.sleep(10)

# Establish ngrok tunnel
public_url = ngrok.connect(8188, "http")
print(f"\n{'='*50}")
print(f"🌐 ComfyUI is running at: {public_url}")
print(f"{'='*50}")
print("⚠️  Keep this cell running to maintain the tunnel")
print("📋 Copy the URL above and paste into your browser")

In [ ]:
import os
from huggingface_hub import login
from google.colab import userdata

# Load token from Colab secrets
hf_token = userdata.get("HF_call")

# Login programmatically
login(token=hf_token)

print("✅ Hugging Face login successful")


**z-image_bf model download**

In [ ]:


REPO_ID = "Comfy-Org/z_image"
SUBFOLDER = "split_files/diffusion_models"
FILENAME = "z_image_bf16.safetensors"
TARGET_DIR = "/content/ComfyUI/models/diffusion_models"

# Create directory
os.makedirs(TARGET_DIR, exist_ok=True)

# Direct download URL
url = f"https://huggingface.co/{REPO_ID}/resolve/main/{SUBFOLDER}/{FILENAME}"
target_path = f"{TARGET_DIR}/{FILENAME}"

!wget -c {url} -O {target_path} --progress=bar:force

print(f"\n✅ Downloaded to: {target_path}")

**encode_qen**

In [ ]:
REPO_ID = "Comfy-Org/z_image_turbo"
FILENAME = "split_files/text_encoders/qwen_3_4b.safetensors"
TARGET_DIR = "/content/ComfyUI/models/text_encoders"

os.makedirs(TARGET_DIR, exist_ok=True)

!huggingface-cli download {REPO_ID} {FILENAME} \
  --local-dir {TARGET_DIR} \
  --local-dir-use-symlinks False

print(f"\n✅ Downloaded to: {TARGET_DIR}/qwen_3_4b.safetensors")

**Vae**

In [ ]:
import os

REPO_ID = "Comfy-Org/z_image_turbo"
FILENAME = "split_files/vae/ae.safetensors"
TARGET_DIR = "/content/ComfyUI/models/vae"

os.makedirs(TARGET_DIR, exist_ok=True)

# Fixed: removed --local-dir-use-symlinks
!hf download {REPO_ID} {FILENAME} --local-dir {TARGET_DIR}

print(f"\n✅ Download complete")
print(f"📂 Location: {TARGET_DIR}/ae.safetensors")
print(f"📂 Location: {TARGET_DIR}/ae.safetensors")